### Installation
For linux:
- install nupack and add 'export NUPACKHOME=/path/to/nupack3.2.1' to ~/.bashrc
- test that Nupack is in path with "echo $NUPACKHOME"
- run a Python 2.7 environment with the following additional packages installed:
 - matplotlib 2.2.5
 - Bio 1.76


# Orthogonal sequence optimizer
## USER INPUT:
- add sequences to be optimized as a dictionary of strings
- add sequences to be fixed (and avoided during optimization) as a dictionary of strings
- set the global physical parameters temperature, sodium concentration (mM), and magnesium concentration (mM)

In [2]:
inp = {
    "p_red" : "CCCTACCTCACCTCACCTCC",
    "GR_OE" : "CCTCCACTCCCACTCCACCT",
    "GR_overlap" : "ACATAACAGAAACAGAAGCA",
    "Tar_overlap" : "CGTGGTAGGTAGGGTGAGGG",
    "p_yel" : "CCCTACCTCCACCTCCACCT",
    "BY_OE" : "ACCTCACTCCACTCCTACCC",
    "p_tar" : "CGCTTGTTCTTGTCTTGTCT",
    
}

fix = {
    "polyg" : "GGGGGGGGGGGGGG",
    "polyt" : "TTTTTTTTTTTTTT",
    "illumina1" : "CAAGCAGAAGAC",
    "illumina2" : "CAAGTTGTCA",
    "illumina3" : "GGCATACGAGAT",
    "illumina4" : "GTCTCGTGGGCTCGGAG",
    "illumina5" : "ATGTGTATAAGAGACAG",
    "illumina6" : "GTGTAGATCTCGG",
    "illumina7" : "TCATTGCACG",
    "illumina8" : "TGGTCGCCGTATCATT",
    "illumina9" : "CTGTCTCTTATACA",
   "illumina10" : "CATCTGACGCTGCCGACGA"    
}
xtr = {
        "bc_r1" : "ACCTCCACCT",
        "bc_ri" : "CCACC",
        "bc_r2" : "TCACTACTCA",
    
        "GR_evL" : "CACTA",
        "GR_evT" : "ACCTA",
    
        "bc_y1" : "CACCTCACCT",
        "bc_yi" : "CCACT",
        "bc_y2" : "ACTCCCACTC",
    
        "BY_evL" : "TACTT",
        "BY_evT" : "TACTA",
    
    "bc_tar" : "TGTTCGTGTCTTGTTCTTGT"
}


global T, magnesium, sodium
T = 60
magnesium = .2
sodium = 1

In [3]:
import imp
from sequence_analysis_lib import sequence_analysis_lib as sq
import pynupack as nu

In [4]:
imp.reload(sq)


### run the optimizer with small chance of reversion to escape local minima ###
### this means that sometimes a change that worsens the score will occur    ###
new_domains_coarse = sq.optimize_domains_quick(inp,
 fix,
 iterations=201, 
 algorithm_reversion_probability= 0.05, 
 plot_interval=50,
 file_prefix = "coarse",
 T = T, magnesium = magnesium, sodium=sodium)

### run the optimizer without reversion - only improvements accepted ###
inp = sq.optimize_domains_quick(new_domains_coarse,
 fix,
 iterations=51, 
 algorithm_reversion_probability= 0.0, 
 plot_interval=50,
 file_prefix = "fine",
 T = T, magnesium = magnesium, sodium=sodium)

strands_dict = {
"GR_amp_s" : inp["p_red"]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"]  ,
"GR_OEL_s" : sq.prime(inp["GR_overlap"])+sq.prime(xtr["GR_evL"])+sq.prime(inp["GR_OE"]),
"GR_OET_s" : inp["Tar_overlap"]+sq.prime(xtr["GR_evT"])+sq.prime(inp["GR_OE"]),
"GR_monL_s" : inp["p_red"]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"]+xtr["GR_evL"]+inp["GR_overlap"],
"GR_monT_s" : inp["p_red"]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"]+xtr["GR_evT"]+sq.prime(inp["Tar_overlap"]),
"BY_amp_s" : inp["p_yel"]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"],
"BY_OEL_s" :inp["GR_overlap"]+sq.prime(xtr["BY_evL"])+sq.prime(inp["BY_OE"]),
"BY_OET_s" : inp["Tar_overlap"]+sq.prime(xtr["BY_evT"])+sq.prime(inp["BY_OE"]),
"BY_monL_s" :inp["p_yel"]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"]+xtr["BY_evL"]+sq.prime(inp["GR_overlap"]),
"BY_monT_s" :inp["p_yel"]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"]+xtr["BY_evT"]+sq.prime(inp["Tar_overlap"]),
"Tar_mon_s" :  inp["p_tar"]+xtr["bc_tar"]+inp["Tar_overlap"],
"full_loc_r_s" :  inp["p_red"]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"]+xtr["GR_evL"]+inp["GR_overlap"]+sq.prime(xtr["BY_evL"])+ sq.prime(inp["BY_OE"])+ sq.prime(xtr["bc_y2"])+ sq.prime(xtr["bc_yi"])+ sq.prime(xtr["bc_y1"])+ sq.prime(inp["p_yel"]),
"full_loc_y_s" :inp["p_yel"]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"]+xtr["BY_evL"]+sq.prime(inp["GR_overlap"])+sq.prime(xtr["GR_evL"])+sq.prime(inp["GR_OE"])+sq.prime(xtr["bc_r2"])+sq.prime(xtr["bc_ri"])+sq.prime(xtr["bc_r1"])+sq.prime(inp["p_red"]),
"full_tar_r_s" :inp["p_red"]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"]+xtr["GR_evT"]+sq.prime(inp["Tar_overlap"])+sq.prime(xtr["bc_tar"])+sq.prime(inp["p_tar"]),
"full_tar_y_s" :inp["p_yel"]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"]+xtr["BY_evT"]+sq.prime(inp["Tar_overlap"])+sq.prime(xtr["bc_tar"])+sq.prime(inp["p_tar"]),
"full_tar_tarR_s" :inp["p_tar"]+xtr["bc_tar"]+inp["Tar_overlap"]+sq.prime(xtr["GR_evT"])+sq.prime(inp["GR_OE"])+sq.prime(xtr["bc_r2"])+sq.prime(xtr["bc_ri"])+sq.prime(xtr["bc_r1"])+sq.prime(inp["p_red"]),
"full_tar_tarY_s" :inp["p_tar"]+xtr["bc_tar"]+inp["Tar_overlap"]+sq.prime(xtr["BY_evT"])+sq.prime(inp["BY_OE"])+sq.prime(xtr["bc_y2"])+sq.prime(xtr["bc_yi"])+sq.prime(xtr["bc_y1"])+sq.prime(inp["p_yel"]),
    }
print(strands_dict.values())

(0, 768.731, 482.8341999999999)
('new variable domains: ', ['CCCTACCTCCACCTCCACCT', 'CACTACCTCACCTCACCTCC', 'ACATAACAGAAACAGAAGCA', 'CGTGGTAGGTAGGGTGAGGG', 'CCTCCACTCCCACTCCACCT', 'CGCTTGTTCTTGTCTTGTCT', 'ACCTCACTCCACTCCTACCC'])
(1, 761.6275, 728.2832000000001)
(2, 759.7062999999999, 724.4413)
(3, 787.278, 800.3581999999999)
(4, 423.6342000000001, 417.7710000000001)
(5, 744.5119000000001, 1063.4434)
(6, 785.7552, 758.4952000000001)
(7, 481.60339999999997, 467.86699999999996)
(8, 731.6528000000001, 434.432)
(9, 728.0062, 698.6165999999998)
(10, 786.8298000000001, 769.6353999999999)
(11, 422.454, 433.4320000000001)
(12, 1068.4474, 1133.3186)
(13, 746.2909000000001, 781.8149999999999)
(14, 471.8094, 518.2622)
(15, 436.0969999999999, 469.694)
(16, 698.5925999999998, 725.293)
(17, 769.6353999999999, 769.9073000000002)
(18, 422.454, 427.8565999999999)
(19, 1068.4474, 1120.8700000000001)
(20, 746.2909000000001, 782.2291000000001)
(21, 471.8094, 442.659)
(22, 436.0969999999999, 436.80600000000